In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
df = pd.read_csv('timeseries_data.csv')

In [35]:
# Assuming 'df' is your DataFrame containing the data

# Select features and target variable
features = ['GS', 'A', 'CS', 'BPS', 'MP']
target = 'Pts'

# Filter data for the 2023 season
df_2023 = df[df['Season'] == '2023-08-01']

# Split the data into features (X) and target variable (y)
X = df[features]
y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, player_names_train, player_names_test = train_test_split(
    X, y, df['Player Name'], test_size=1, random_state=42
)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Use the scaler trained on the training set to transform the 2023 season data
X_test_2023_scaled = scaler.transform(df_2023[features])

# Create a Linear Regression model
model = LinearRegression()

# Train the model on historical data
model.fit(X_train_scaled, y_train)

# Make predictions on the 2023 season data
y_pred_2023 = model.predict(X_test_2023_scaled)


In [38]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [40]:
# Create a DataFrame to display the results for the 2023 season
results_2023_df = pd.DataFrame({
    'Player Name': df_2023['Player Name'],
    'Actual Pts': df_2023['Pts'],
    'Predicted Pts': y_pred_2023.round().astype(int) # Use rounded and integer predicted points
})

# Reset the index of the DataFrame
results_2023_df = results_2023_df.reset_index(drop=True)

# Display the results for the 2023 season
results_2023_df

,Player Name,Actual Pts,Predicted Pts
0,Salah,127,118
1,Son,113,105
2,Haaland,112,126
3,Watkins,99,104
4,Saka,94,94
5,Mbeumo,87,77
6,Bowen,86,74
7,Gordon,83,85
8,Trippier,82,81
9,J.Alvarez,80,91
